# Google Cloud CMIP6 Public Data: Basic Python Example

This notebooks shows how to query the catalog and load the data using python

In [ ]:
! pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.1 MB/s eta 0:00:00


In [ ]:

! pip install --upgrade xarray zarr gcsfs cftime nc-time-axis


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 9.1 MB/s eta 0:00:00
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5034 sha256=d4a0521960054f9fc0f22e6a4d02641f8296f6725a7bbdf66ac0fc45b2aacbe7
  Stored in directory: /root/.cache/pip/wheels/7f/4e/be/1171b40f43b918087657ec57cf3b81fa1a2e027d8755baa184
Successfully built asciitree
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
    Uninstalling fsspec-2023.6.0:
      Successfully uninstalled fsspec-2023.6.0
  Attempting uninstall: xarray
    Found existing installation: xarray 2023.7.0
    Uninstalling xarray-2023.7.0:
      Successfully uninstalled xarray-2023.7.0
  Attempting uninstall: 

In [ ]:
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs


## Browse Catalog

The data catatalog is stored as a CSV file. Here we read it with Pandas.

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


In [ ]:
set(df.source_id)

{'ACCESS-CM2',
 'ACCESS-ESM1-5',
 'AWI-CM-1-1-MR',
 'AWI-ESM-1-1-LR',
 'BCC-CSM2-HR',
 'BCC-CSM2-MR',
 'BCC-ESM1',
 'CAMS-CSM1-0',
 'CAS-ESM2-0',
 'CESM1-1-CAM5-CMIP5',
 'CESM1-WACCM-SC',
 'CESM2',
 'CESM2-FV2',
 'CESM2-WACCM',
 'CESM2-WACCM-FV2',
 'CIESM',
 'CMCC-CM2-HR4',
 'CMCC-CM2-SR5',
 'CMCC-CM2-VHR4',
 'CMCC-ESM2',
 'CNRM-CM6-1',
 'CNRM-CM6-1-HR',
 'CNRM-ESM2-1',
 'CanESM5',
 'CanESM5-CanOE',
 'E3SM-1-0',
 'E3SM-1-1',
 'E3SM-1-1-ECA',
 'EC-Earth3',
 'EC-Earth3-AerChem',
 'EC-Earth3-CC',
 'EC-Earth3-LR',
 'EC-Earth3-Veg',
 'EC-Earth3-Veg-LR',
 'EC-Earth3P',
 'EC-Earth3P-HR',
 'EC-Earth3P-VHR',
 'ECMWF-IFS-HR',
 'ECMWF-IFS-LR',
 'FGOALS-f3-H',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'FIO-ESM-2-0',
 'GFDL-AM4',
 'GFDL-CM4',
 'GFDL-CM4C192',
 'GFDL-ESM2M',
 'GFDL-ESM4',
 'GFDL-OM4p5B',
 'GISS-E2-1-G',
 'GISS-E2-1-G-CC',
 'GISS-E2-1-H',
 'GISS-E2-2-G',
 'GISS-E2-2-H',
 'HadGEM3-GC31-HM',
 'HadGEM3-GC31-LL',
 'HadGEM3-GC31-LM',
 'HadGEM3-GC31-MM',
 'ICON-ESM-LR',
 'IITM-ESM',
 'INM-CM4-8',
 'I

In [ ]:
gcs = gcsfs.GCSFileSystem(token='anon')

## Load Data

Now we will load a single store using gcsfs, zarr, and xarray.

Not all the models have all the simulations and all the variables we need. So first, let's subset the models that have all the data

In [ ]:
# For each simulation (historical, ssp5858) and each variable (tas, precip) let's find all the available models
df_tas_hist=df.query  ("activity_id=='CMIP'         & table_id == 'Amon' & variable_id == 'tas' & experiment_id=='historical'& member_id=='r1i1p1f1'")
df_tas_ssp585=df.query("activity_id=='ScenarioMIP'  & table_id == 'Amon' & variable_id == 'tas' & experiment_id=='ssp585'    & member_id=='r1i1p1f1'")
df_pr_hist=df.query   ("activity_id=='CMIP'         & table_id == 'Amon' & variable_id == 'pr'  & experiment_id=='historical'& member_id=='r1i1p1f1'")
df_pr_ssp585=df.query ("activity_id=='ScenarioMIP'  & table_id == 'Amon' & variable_id == 'pr'  & experiment_id=='ssp585'    & member_id=='r1i1p1f1'")

#  assemble a set of models (source_id) for each experiment + variable combination
models_tas_hist  =set(df_tas_hist.source_id)    # all models that have temperature in the historical simulations
models_tas_ssp585=set(df_tas_ssp585.source_id)  # all models that have temperature in the ssp585     simulations
models_pr_hist   =set(df_pr_hist.source_id)     # all models that have precip  in the historical simulations
models_pr_ssp585 =set(df_pr_ssp585.source_id)   # all models that have precip in the ssp585 simulations

# let's only keep the models that have all the runs (the intersection of the set)
source_set = set.intersection(models_tas_hist, models_tas_ssp585, models_pr_hist, models_pr_ssp585)
source_list=list(source_set)

source_list.remove('MPI-ESM1-2-LR')
source_list.remove('MPI-ESM1-2-HR')
source_list.remove('KIOST-ESM')
source_list.remove('ACCESS-CM2')
source_list.remove('ACCESS-ESM1-5')
source_list.remove('FGOALS-g3')

#Let's look at how many models have the data for each of these queries
print(len(models_tas_hist))
print(len(models_tas_ssp585))
print(len(models_pr_hist))
print(len(models_pr_ssp585))

#how many models have all the data
print(len(source_list))



55
35
54
34
27


In [ ]:
new_lat = np.arange(-85, 85)
new_lon = np.arange(2,358)

In [ ]:
def load_zarr_dset(df):
  # get the path to a specific zarr store (the first one from the dataframe above)
  zstore = df.zstore.values[-1]

  # create a mutable-mapping-style interface to the store
  mapper = gcs.get_mapper(zstore)

  # open it using xarray and zarr
  ds = xr.open_zarr(mapper, consolidated=True)
  return ds




#Find Regional Precipitation and Temperature in history and predicted
also find global precipiation and temperature in history and predicted

In [ ]:
ds_list = list()


for source in source_list:

    df_hist_tas=df.query  ("activity_id=='CMIP'         & table_id == 'Amon' & variable_id == 'tas' & experiment_id=='historical'& member_id=='r1i1p1f1' & source_id==@source")
    df_ssp585_tas=df.query("activity_id=='ScenarioMIP'  & table_id == 'Amon' & variable_id == 'tas' & experiment_id=='ssp585'    & member_id=='r1i1p1f1' & source_id==@source")
    df_hist_pr=df.query   ("activity_id=='CMIP'         & table_id == 'Amon' & variable_id == 'pr'  & experiment_id=='historical'& member_id=='r1i1p1f1' & source_id==@source")
    df_ssp585_pr=df.query ("activity_id=='ScenarioMIP'  & table_id == 'Amon' & variable_id == 'pr'  & experiment_id=='ssp585'    & member_id=='r1i1p1f1' & source_id==@source")

    ds_hist_tas   =load_zarr_dset(df_hist_tas)
    ds_ssp585_tas =load_zarr_dset(df_ssp585_tas)
    ds_hist_pr    =load_zarr_dset(df_hist_pr)
    ds_ssp585_pr  =load_zarr_dset(df_ssp585_pr)


  #finding the historical precipitation mean from 1980 to 1999 using the newly created xarray dataset
    ds_hist_pr_mean = ds_hist_pr.sel(time=slice("1980", "1999")).mean(dim='time')

  #finding the predicted precipitation mean from 2080 to 2099
    ds_ssp585_pr_mean = ds_ssp585_pr.sel(time=slice("2080", "2099")).mean(dim='time')

  #finding the historical precipitation mean from 1980 to 1999 using the newly created xarray dataset
    ds_hist_tas_mean = ds_hist_tas.sel(time=slice("1980", "1999")).mean(dim='time')

  #finding the predicted precipitation mean from 2080 to 2099
    ds_ssp585_tas_mean = ds_ssp585_tas.sel(time=slice("2080", "2099")).mean(dim='time')



   #load the temperature data
    tas = ds_ssp585_tas.tas.sel(time=slice("2000", "2099"))

  #find the weights of the area of the grid cells
    weights = np.cos(np.deg2rad(tas.lat))

  #subtract to find delta
    ds_delta_pr = ds_ssp585_pr_mean - ds_hist_pr_mean

    ds_delta_tas = ds_ssp585_tas_mean - ds_hist_tas_mean

  #find the mean of the weighted temperature data and add it to the dataset
    dTg = ds_delta_tas.tas.weighted(weights).mean(dim=('lon', 'lat'))

    ds_delta_tas = ds_delta_tas.assign(dTg=dTg)
  # do the same for precip
    dPg = ds_delta_pr.pr.weighted(weights).mean(dim=('lon', 'lat'))
    ds_delta_pr  = ds_delta_pr.assign(dPg=dPg)




  #load the temperature data
    tas_1 = ds_hist_tas.tas.sel(time=slice("1980", "1999"))

    tas_2 = ds_hist_tas.tas.sel(time=slice("1850", "1900"))

  #load precip data
    pr_1 = ds_hist_pr.pr.sel(time=slice("1980", "1999"))

    pr_2 = ds_hist_pr.pr.sel(time=slice("1850", "1900"))

  #find the weights of the area of the grid cells
    weights_1 = np.cos(np.deg2rad(tas_1.lat))

    weights_2 = np.cos(np.deg2rad(tas_2.lat))

  #find the mean of the weighted historical temperature data and subtract the different time periods
    dTg_hist = tas_1.weighted(weights_1).mean(dim=('lon', 'lat', 'time')) - tas_2.weighted(weights_2).mean(dim=('lon', 'lat', 'time'))

  #do the same for precip
    dPg_hist = pr_1.weighted(weights_1).mean(dim=('lon', 'lat', 'time')) - pr_2.weighted(weights_2).mean(dim=('lon', 'lat', 'time'))



  #add historical temp and precip to the data set
    ds_delta_tas = ds_delta_tas.assign(dTg_hist=dTg_hist)

    ds_delta_tas = ds_delta_tas.assign(dPg_hist=dPg_hist)

  #merge the delta datasets together
    ds_merged = xr.merge([ds_delta_pr, ds_delta_tas],compat='override')


  #interpolate the data to a common (1x1) grid
    ds_interp = ds_merged.interp(lat=new_lat, lon= new_lon)


  #Append the data to list
    ds_list.append(ds_interp)


#merge the data into one dataset

ds_all = xr.concat([*ds_list], dim = 'model',coords='minimal',compat='override')


ds_all

KeyboardInterrupt: 

In [ ]:
ds_all.load()

In [ ]:
#let's clean this up a bit
ds_all=ds_all.drop(['height','lat_bnds','lon_bnds','bnds'])
ds_all = ds_all.assign(model = source_list)

ds_all

<xarray.Dataset>
Dimensions:   (model: 27, lat: 170, lon: 356)
Coordinates:
  * lat       (lat) int64 -85 -84 -83 -82 -81 -80 -79 ... 78 79 80 81 82 83 84
  * lon       (lon) int64 2 3 4 5 6 7 8 9 10 ... 350 351 352 353 354 355 356 357
  * model     (model) <U16 'KACE-1-0-G' 'E3SM-1-1' ... 'EC-Earth3' 'GFDL-ESM4'
Data variables:
    pr        (model, lat, lon) float64 1.118e-06 1.108e-06 ... 2.714e-06
    dPg       (model) float64 3.417e-06 4.34e-06 ... 3.221e-06 1.128e-06
    tas       (model, lat, lon) float64 6.087 6.095 6.1 ... 9.172 9.165 9.159
    dTg       (model) float64 4.963 6.973 3.876 4.489 ... 4.693 4.52 4.97 3.417
    dTg_hist  (model) float64 0.7382 0.01576 0.7192 ... 0.5716 0.2141 0.2583
    dPg_hist  (model) float64 2.179e-07 -6.886e-07 ... -7.692e-08 -3.488e-07

#Find Correlations
(xTTg, xTTG_hist, xPTG, xPTG_hist)

In [ ]:
#correlate changes in global temp of 21st century to changes in regional temp
xTTg = xr.corr(ds_all.tas, ds_all.dTg,dim='model')
#add to dataset
ds_all = ds_all.assign(xTTg=xTTg)

#correlate changes in global temp of 21st century to changes in regional precip
xPTg = xr.corr(ds_all.pr, ds_all.dTg,dim='model')
#add to dataset
ds_all = ds_all.assign(xPTg=xPTg)

#correlate changes in global temp of 21st century to changes in regional precip
xPPg = xr.corr(ds_all.pr, ds_all.dPg,dim='model')
#add to dataset
ds_all = ds_all.assign(xPPg=xPPg)

#correlate historical changes in global temp to changes in regional temp
xTTg_hist = xr.corr(ds_all.tas, ds_all.dTg_hist,dim='model')
#add to dataset
ds_all = ds_all.assign(xTTg_hist=xTTg_hist)

#correlate historical changes in global temp to changes in regional precip
xPTg_hist = xr.corr(ds_all.pr, ds_all.dTg_hist,dim='model')
#add to dataset
ds_all = ds_all.assign(xPTg_hist=xPTg_hist)

#correlate historical changes in global temp to changes in regional precip
xPPg_hist = xr.corr(ds_all.pr, ds_all.dPg_hist,dim='model')
#add to dataset
ds_all = ds_all.assign(xPPg_hist=xPPg_hist)

ds_all

<xarray.Dataset>
Dimensions:    (model: 27, lat: 170, lon: 356)
Coordinates:
  * lat        (lat) int64 -85 -84 -83 -82 -81 -80 -79 ... 78 79 80 81 82 83 84
  * lon        (lon) int64 2 3 4 5 6 7 8 9 ... 350 351 352 353 354 355 356 357
  * model      (model) <U16 'KACE-1-0-G' 'E3SM-1-1' ... 'EC-Earth3' 'GFDL-ESM4'
Data variables:
    pr         (model, lat, lon) float64 1.118e-06 1.108e-06 ... 2.714e-06
    dPg        (model) float64 3.417e-06 4.34e-06 ... 3.221e-06 1.128e-06
    tas        (model, lat, lon) float64 6.087 6.095 6.1 ... 9.172 9.165 9.159
    dTg        (model) float64 4.963 6.973 3.876 4.489 ... 4.693 4.52 4.97 3.417
    dTg_hist   (model) float64 0.7382 0.01576 0.7192 ... 0.5716 0.2141 0.2583
    dPg_hist   (model) float64 2.179e-07 -6.886e-07 ... -7.692e-08 -3.488e-07
    xTTg       (lat, lon) float64 0.8255 0.8266 0.8278 ... 0.8552 0.8547 0.8527
    xPTg       (lat, lon) float64 0.7703 0.7665 0.7645 ... 0.7302 0.7257 0.7227
    xPPg       (lat, lon) float64 0.7225 0.7146 0.7083 ... 0.7189 0.7319 0.7438
    xTTg_hist  (lat, lon) float64 -0.0793 -0.07699 -0.07516 ... 0.1356 0.137
    xPTg_hist  (lat, lon) float64 -0.07284 -0.05707 -0.03966 ... 0.2355 0.2209
    xPPg_hist  (lat, lon) float64 -0.3259 -0.3155 -0.3036 ... 0.1155 0.09743

In [ ]:
ds_all.to_netcdf("CMIP6_pre-processed_data.nc", mode = 'w', format = "NETCDF4", engine = 'netcdf4')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
